### Ejercicios


**1. Implementación de un autoencoder básico para compresión de datos**  

*Enunciado detallado:*  
Construye un autoencoder simple para comprimir y reconstruir imágenes (por ejemplo, utilizando el dataset MNIST). La red debe estar compuesta por dos partes:  
- **Encoder:** Reduce la dimensión de la imagen a un espacio latente de menor dimensionalidad.  
- **Decoder:** Reconstruye la imagen original a partir de la representación latente.  

El ejercicio implica ajustar la arquitectura (número de capas y neuronas), elegir funciones de activación (como ReLU o Sigmoid) y entrenar el modelo para minimizar la diferencia entre la imagen original y la reconstruida (por ejemplo, usando MSELoss).

*Código de referencia:*
```python
import torch
import torch.nn as nn

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # Encoder: de 28x28 a un vector latente de tamaño 32
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32)
        )
        # Decoder: de vector latente de tamaño 32 a 28x28
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 28*28),
            nn.Sigmoid()  # Para que la salida esté en el rango [0,1]
        )
    
    def forward(self, x):
        # Aplanar la imagen de entrada
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        out = self.decoder(z)
        # Reconstruir la imagen original
        out = out.view(x.size(0), 1, 28, 28)
        return out

# Ejemplo de creación del modelo
modelo = Autoencoder()
print(modelo)
```


**2. Desarrollo de un autoencoder variacional (VAE) para generación de imágenes**

*Enunciado detallado:*  
Implementa un VAE que, además de comprimir las imágenes, aprenda una distribución en el espacio latente. El ejercicio se centra en:  
- Definir un encoder que genere los parámetros (media y log-varianza) de la distribución latente.  
- Realizar el muestreo utilizando la "reparametrización" para permitir el flujo de gradiente.  
- Diseñar un decoder que genere imágenes a partir de muestras del espacio latente.  

Evalúa la capacidad del VAE para generar nuevas imágenes a partir de la distribución latente aprendida.

*Código de referencia:*
```python
import torch
import torch.nn as nn
import torch.nn.functional as F

class VAE(nn.Module):
    def __init__(self, latent_dim=20):
        super(VAE, self).__init__()
        # Encoder
        self.fc1 = nn.Linear(28*28, 400)
        self.fc_mu = nn.Linear(400, latent_dim)
        self.fc_logvar = nn.Linear(400, latent_dim)
        # Decoder
        self.fc2 = nn.Linear(latent_dim, 400)
        self.fc3 = nn.Linear(400, 28*28)
    
    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        mu = self.fc_mu(h1)
        logvar = self.fc_logvar(h1)
        return mu, logvar
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def decode(self, z):
        h3 = F.relu(self.fc2(z))
        return torch.sigmoid(self.fc3(h3))
    
    def forward(self, x):
        x = x.view(-1, 28*28)
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        reconstruction = self.decode(z)
        reconstruction = reconstruction.view(-1, 1, 28, 28)
        return reconstruction, mu, logvar

# Ejemplo de creación del modelo
vae = VAE(latent_dim=20)
print(vae)
```

**3. Aplicación de autoencoders en reducción de dimensionalidad y visualización** 

*Enunciado detallado:*  
Utiliza un autoencoder (similar al del ejercicio 1) para reducir la dimensionalidad de un dataset de imágenes o datos de alta dimensión. Una vez entrenado, extrae la representación latente y utiliza técnicas de visualización (por ejemplo, t-SNE o PCA) para observar la agrupación y la estructura interna de los datos.  
El ejercicio permite explorar cómo el autoencoder aprende representaciones compactas y cómo estas pueden ayudar a identificar clusters o patrones.

*Código de referencia (extracción de la representación latente):*
```python
import torch
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Suponiendo que 'model' es el autoencoder entrenado del ejercicio 1 y 'data_loader' contiene imágenes
def get_latent_representations(model, data_loader):
    model.eval()
    latents = []
    labels = []
    with torch.no_grad():
        for images, lbl in data_loader:
            images = images.to(next(model.parameters()).device)
            # Aplanar imágenes y pasar por el encoder
            images = images.view(images.size(0), -1)
            latent = model.encoder(images)
            latents.append(latent.cpu())
            labels.extend(lbl.numpy())
    return torch.cat(latents, dim=0), labels

# Una vez obtenidas las representaciones latentes:
# latents, labels = get_latent_representations(model, data_loader)
# tsne = TSNE(n_components=2)
# latents_2d = tsne.fit_transform(latents)
# plt.scatter(latents_2d[:, 0], latents_2d[:, 1], c=labels, cmap='viridis')
# plt.show()
```

**4. Implementación del mecanismo de atención "scaled dot-product"**

*Enunciado detallado:*  
Desarrolla una función en PyTorch que implemente el mecanismo de atención "scaled dot-product". La función debe recibir como entrada tensores de consulta (query), llave (key) y valor (value), calcular la puntuación mediante el producto escalar, aplicar el escalado por la raíz cuadrada de la dimensión y, finalmente, usar softmax para obtener los pesos de atención.  
Este ejercicio es clave para entender cómo se ponderan las relaciones entre diferentes elementos en una secuencia.

*Código de referencia:*
```python
import torch
import torch.nn.functional as F
import math

def scaled_dot_product_attention(query, key, value):
    # query, key, value: [batch_size, num_heads, seq_len, d_k]
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    attn = F.softmax(scores, dim=-1)
    output = torch.matmul(attn, value)
    return output, attn

# Ejemplo de uso:
batch_size, num_heads, seq_len, d_k = 2, 4, 10, 16
query = torch.randn(batch_size, num_heads, seq_len, d_k)
key   = torch.randn(batch_size, num_heads, seq_len, d_k)
value = torch.randn(batch_size, num_heads, seq_len, d_k)

output, attention = scaled_dot_product_attention(query, key, value)
print("Output shape:", output.shape)
print("Attention shape:", attention.shape)
```

**5. Construcción de un módulo de atención multi-cabecera en un modelo transformer**  
*Enunciado detallado:*  
Implementa un módulo de atención multi-cabecera que divida la entrada en varias "cabeceras", aplique el mecanismo de atención en cada una y luego combine los resultados.  

Aspectos a considerar:  
- Proyectar las entradas a espacios de query, key y value para cada cabecera.  
- Concatenar las salidas de cada cabecera y aplicar una proyección final.  
Este módulo es esencial para entender cómo el transformer procesa información de forma paralela en diferentes subespacios.

*Código de referencia:*
```python
import torch
import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert embed_dim % num_heads == 0, "El embedding debe ser divisible por el número de cabecera"
        self.num_heads = num_heads
        self.d_k = embed_dim // num_heads
        
        self.q_linear = nn.Linear(embed_dim, embed_dim)
        self.k_linear = nn.Linear(embed_dim, embed_dim)
        self.v_linear = nn.Linear(embed_dim, embed_dim)
        self.out_linear = nn.Linear(embed_dim, embed_dim)
    
    def forward(self, x):
        batch_size, seq_len, embed_dim = x.size()
        # Proyección lineal
        Q = self.q_linear(x)
        K = self.k_linear(x)
        V = self.v_linear(x)
        
        # Dividir en múltiples cabecera y reorganizar dimensiones
        Q = Q.view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1,2)
        K = K.view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1,2)
        V = V.view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1,2)
        
        # Aplicar atención escalada (utilizando la función del ejercicio 4)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        attn = torch.softmax(scores, dim=-1)
        context = torch.matmul(attn, V)
        
        # Reorganizar la salida
        context = context.transpose(1,2).contiguous().view(batch_size, seq_len, embed_dim)
        output = self.out_linear(context)
        return output, attn

# Ejemplo de uso:
batch_size, seq_len, embed_dim, num_heads = 2, 15, 64, 8
x = torch.randn(batch_size, seq_len, embed_dim)
mha = MultiHeadAttention(embed_dim, num_heads)
output, attn = mha(x)
print("Output shape:", output.shape)
```

**6. Entrenamiento y escalado de un modelo transformer para modelado de lenguaje**  
*Enunciado detallado:*  
Construye un modelo transformer para modelado de lenguaje (por ejemplo, predicción de la siguiente palabra o traducción). Este ejercicio involucra:  
- Definir un encoder (y/o decoder) basado en bloques de atención multicabecera, capas feedforward y mecanismos de normalización.  
- Entrenar el modelo con un corpus de texto, experimentando con distintos tamaños (número de capas, cabeceras de atención, etc.) para observar cómo afecta la capacidad del modelo.  
- Evaluar el rendimiento mediante métricas comunes en NLP, como la pérdida de entropía cruzada.

*Código de referencia (usando el módulo Transformer de PyTorch):*
```python
import torch
import torch.nn as nn

# Parámetros del modelo
vocab_size = 10000
embed_dim = 512
num_heads = 8
num_layers = 6
seq_length = 20

class TransformerModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, num_layers):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.positional_encoding = nn.Parameter(torch.zeros(1, seq_length, embed_dim))
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(embed_dim, vocab_size)
    
    def forward(self, x):
        # x: [batch_size, seq_length]
        x = self.embedding(x) + self.positional_encoding
        x = x.transpose(0,1)  # Transformer espera [seq_length, batch_size, embed_dim]
        x = self.transformer_encoder(x)
        x = x.transpose(0,1)
        logits = self.fc_out(x)
        return logits

# Ejemplo de uso:
modelo = TransformerModel(vocab_size, embed_dim, num_heads, num_layers)
dummy_input = torch.randint(0, vocab_size, (8, seq_length))
logits = modelo(dummy_input)
print("Logits shape:", logits.shape)
```

**7. Comparación de variantes del mecanismo de atención en tareas de NLP**  
*Enunciado detallado:*  
Diseña e implementa al menos dos variantes del mecanismo de atención (por ejemplo, atención escalada dot-product vs. atención local o autoregresiva) y aplícalas a una tarea de NLP como clasificación de texto o análisis de sentimientos.  
El ejercicio debe incluir:  
- La implementación de las variantes, preferiblemente en módulos reutilizables.  
- Un estudio comparativo donde se entrene cada variante bajo las mismas condiciones y se analicen métricas de desempeño (p. ej., precisión, pérdida).  
- Reflexiones sobre cómo cada variante maneja la dependencia entre elementos de la secuencia.

*Código de referencia (esquemático de dos variantes de atención):*
```python
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

# Variante 1: Atención escalada dot-product (igual que en el ejercicio 4)
def scaled_dot_product_attention(query, key, value):
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    attn = F.softmax(scores, dim=-1)
    output = torch.matmul(attn, value)
    return output, attn

# Variante 2: Atención local (limitando la atención a una ventana)
def local_attention(query, key, value, window_size=5):
    batch_size, num_heads, seq_len, d_k = query.size()
    outputs = []
    all_attn = []
    for i in range(seq_len):
        # Definir límites de la ventana
        start = max(0, i - window_size)
        end = min(seq_len, i + window_size + 1)
        # Extraer las partes relevantes
        q = query[:, :, i:i+1, :]
        k_slice = key[:, :, start:end, :]
        v_slice = value[:, :, start:end, :]
        # Calcular atención para la posición i
        scores = torch.matmul(q, k_slice.transpose(-2, -1)) / math.sqrt(d_k)
        attn = F.softmax(scores, dim=-1)
        output = torch.matmul(attn, v_slice)
        outputs.append(output)
        all_attn.append(attn)
    outputs = torch.cat(outputs, dim=2)
    all_attn = torch.cat(all_attn, dim=2)
    return outputs, all_attn

# Ejemplo de uso para ambas variantes:
batch_size, num_heads, seq_len, d_k = 2, 4, 10, 16
query = torch.randn(batch_size, num_heads, seq_len, d_k)
key   = torch.randn(batch_size, num_heads, seq_len, d_k)
value = torch.randn(batch_size, num_heads, seq_len, d_k)

out1, attn1 = scaled_dot_product_attention(query, key, value)
out2, attn2 = local_attention(query, key, value, window_size=3)

print("Scaled dot-product output shape:", out1.shape)
print("Local attention output shape:", out2.shape)
```


**8. Implementación de positional encoding en PyTorch**  

*Enunciado detallado:*  
Desarrolla una función que implemente la codificación posicional sinusoidal, tal como se describe en el paper original de transformers. Esta codificación deberá sumarse a las embeddings de entrada para inyectar información posicional en el modelo.  
*Objetivos:*  
- Comprender la importancia de la codificación posicional.  
- Implementar el cálculo de senos y cosenos en diferentes frecuencias.  
- Integrar la codificación posicional a un pipeline de embeddings.  

*Código de referencia:*
```python
import torch
import math

def positional_encoding(seq_len, embed_dim):
    pe = torch.zeros(seq_len, embed_dim)
    position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-math.log(10000.0) / embed_dim))
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    return pe

# Ejemplo de uso:
seq_len = 50
embed_dim = 512
pe = positional_encoding(seq_len, embed_dim)
print("Positional Encoding shape:", pe.shape)
```


**9. Implementación de un bloque de transformer encoder desde cero**  

*Enunciado detallado:*  
Construye un bloque de encoder de transformer que incluya los siguientes componentes:  

- Módulo de atención multicabecera (con proyecciones lineales y atención escalada dot-product).  
- Capa feedforward con al menos dos capas lineales y función de activación (por ejemplo, ReLU).  
- Conexiones residuales y normalización de capas (LayerNorm) en ambos sub-bloques.  

*Objetivos:*  
- Entender la estructura interna de un bloque encoder.  
- Implementar el mecanismo de residual connection y normalización para estabilizar el entrenamiento.  

*Código de referencia:*
```python
import torch
import torch.nn as nn
import math

class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert embed_dim % num_heads == 0, "El embedding debe ser divisible por el número de cabecera"
        self.num_heads = num_heads
        self.d_k = embed_dim // num_heads
        
        self.q_linear = nn.Linear(embed_dim, embed_dim)
        self.k_linear = nn.Linear(embed_dim, embed_dim)
        self.v_linear = nn.Linear(embed_dim, embed_dim)
        self.out_linear = nn.Linear(embed_dim, embed_dim)
    
    def forward(self, x):
        batch_size, seq_len, embed_dim = x.size()
        Q = self.q_linear(x)
        K = self.k_linear(x)
        V = self.v_linear(x)
        # Reorganizar para múltiples cabeceras
        Q = Q.view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1,2)
        K = K.view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1,2)
        V = V.view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1,2)
        
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        attn = torch.softmax(scores, dim=-1)
        context = torch.matmul(attn, V)
        context = context.transpose(1,2).contiguous().view(batch_size, seq_len, embed_dim)
        output = self.out_linear(context)
        return output

class TransformerEncoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_hidden_dim, dropout=0.1):
        super(TransformerEncoderBlock, self).__init__()
        self.mha = MultiHeadAttention(embed_dim, num_heads)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)
        # Capa feedforward
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, ff_hidden_dim),
            nn.ReLU(),
            nn.Linear(ff_hidden_dim, embed_dim)
        )
    
    def forward(self, x):
        # Primer sub-bloque: atención multicabecera + residual + normalización
        attn_output = self.mha(x)
        x = self.norm1(x + self.dropout(attn_output))
        # Segundo sub-bloque: capa feedforward + residual + normalización
        ff_output = self.ff(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

# Ejemplo de uso:
batch_size, seq_len, embed_dim = 2, 10, 64
num_heads = 8
ff_hidden_dim = 256
dummy_input = torch.randn(batch_size, seq_len, embed_dim)
encoder_block = TransformerEncoderBlock(embed_dim, num_heads, ff_hidden_dim)
output = encoder_block(dummy_input)
print("Encoder block output shape:", output.shape)
```


**10. Construcción de un modelo transformer completo para traducción**  

*Enunciado detallado:*

Utilizando el módulo `nn.Transformer` de PyTorch, crea un modelo completo de encoder-decoder para una tarea de traducción automática. El ejercicio incluye:  
- Definir embeddings para el idioma de origen y destino.  
- Aplicar codificación posicional a ambas entradas.  
- Configurar el transformer con parámetros ajustables (número de capas, cabeceras, etc.).  
- Implementar la función de entrenamiento que utilice la entropía cruzada como pérdida.  

*Objetivos:*  
- Integrar encoder y decoder en un modelo end-to-end.  
- Entender cómo se gestionan las máscaras para el decoder en tareas de traducción.  

*Código de referencia:*
```python
import torch
import torch.nn as nn

class SimpleTransformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, embed_dim, num_heads, num_layers, dropout=0.1, max_seq_len=50):
        super(SimpleTransformer, self).__init__()
        self.src_embedding = nn.Embedding(src_vocab_size, embed_dim)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, embed_dim)
        self.positional_encoding = positional_encoding(max_seq_len, embed_dim).unsqueeze(0)
        
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dropout=dropout)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        decoder_layer = nn.TransformerDecoderLayer(d_model=embed_dim, nhead=num_heads, dropout=dropout)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)
        
        self.fc_out = nn.Linear(embed_dim, tgt_vocab_size)
    
    def forward(self, src, tgt):
        # src, tgt: [batch_size, seq_len]
        src_emb = self.src_embedding(src) + self.positional_encoding[:, :src.size(1), :]
        tgt_emb = self.tgt_embedding(tgt) + self.positional_encoding[:, :tgt.size(1), :]
        
        # Transformer espera entrada [seq_len, batch_size, embed_dim]
        src_emb = src_emb.transpose(0,1)
        tgt_emb = tgt_emb.transpose(0,1)
        
        memory = self.encoder(src_emb)
        output = self.decoder(tgt_emb, memory)
        output = self.fc_out(output)
        return output

# Ejemplo de uso:
src_vocab_size = 5000
tgt_vocab_size = 5000
modelo = SimpleTransformer(src_vocab_size, tgt_vocab_size, embed_dim=256, num_heads=8, num_layers=3)
dummy_src = torch.randint(0, src_vocab_size, (8, 20))
dummy_tgt = torch.randint(0, tgt_vocab_size, (8, 20))
output = modelo(dummy_src, dummy_tgt)
print("Transformer output shape:", output.shape)
```


**11. Ajuste fino de un modelo transformer preentrenado para clasificación de texto**  
*Enunciado detallado:*  
Utiliza un modelo Transformer preentrenado (por ejemplo, BERT o similar disponible en Hugging Face) y realiza el ajuste fino para una tarea de clasificación de texto. El ejercicio debe cubrir:  
- Carga del modelo preentrenado.  
- Adición de una capa de salida para clasificación.  
- Preparación de un dataset de texto y tokenización adecuada.  
- Entrenamiento y evaluación del modelo en la tarea objetivo.  

*Objetivos:*  
- Aprender a transferir conocimiento de un modelo preentrenado a tareas específicas.  
- Comprender las técnicas de ajuste fino (fine-tuning) en modelos de lenguaje.  

*Código de referencia (esquemático):*
```python
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Cargar el tokenizador y modelo preentrenado
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
modelo = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Ejemplo de preparación de datos
sentences = ["This is a positive example.", "This is a negative example."]
inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=50)

# Ejemplo de paso de entrenamiento
labels = torch.tensor([1, 0]).unsqueeze(0)  # Ajustar dimensiones según sea necesario
outputs = modelo(**inputs, labels=labels.squeeze())
loss = outputs.loss
logits = outputs.logits
print("Loss:", loss.item())
```

**12. Estudio de la escalabilidad de transformers en tareas de modelado de lenguaje**  

*Enunciado detallado:*  
Investiga cómo varían el rendimiento y el tiempo de entrenamiento de un modelo transformer al modificar su arquitectura. Para ello, diseña un experimento en el que:  
- Se entrene un modelo transformer para modelado de lenguaje (por ejemplo, predicción de la siguiente palabra).  
- Se varíen parámetros como el número de capas del encoder, la cantidad de cabeceras en la atención y el tamaño de las capas feedforward.  
- Se registren métricas de desempeño (pérdida, perplexity) y tiempos de entrenamiento, para comparar la escalabilidad y eficiencia del modelo.  

*Objetivos:*  
- Analizar el trade-off entre la complejidad del modelo y su desempeño en tareas de NLP.  
- Experimentar con hiperparámetros y comprender su impacto en el entrenamiento.  

*Código de referencia (estructura básica de experimento):*
```python
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# Aquí se esquematiza la carga de datos, tokenización y definición de un modelo transformer
# La idea es entrenar con diferentes configuraciones y registrar resultados.

def train_model(model, data_loader, epochs=5):
    optimizer = optim.Adam(model.parameters(), lr=0.0005)
    criterion = nn.CrossEntropyLoss()
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in data_loader:
            optimizer.zero_grad()
            # Supongamos que 'src' es la secuencia de entrada y 'tgt' la secuencia a predecir
            src, tgt = batch['src'], batch['tgt']
            output = model(src, tgt[:, :-1])
            loss = criterion(output.view(-1, output.size(-1)), tgt[:, 1:].reshape(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoca {epoch+1}, Loss: {total_loss/len(data_loader)}")

# Ejemplo: cambiar número de capas y cabezas
# Se deben definir distintos modelos con configuraciones variantes y evaluar su desempeño.
```

In [ ]:
## Tus respuestas